# Día 5: Automatización de un reporte en Excel

### Objetivo:

Aprender a generar automaticamente un archivo de Excel a partir de datos con:

*   Múltiples hojas
*   Formato aplicado (negritas, filtros, anchos)
*   Tablas resumen
*   Nombre del archivo con fecha

#### Situación 1:

Recibes datos del departamento de ventas por producto, necesitas crear un excel com el resumen de ventas por producto, graficarlo, guardarlo con un nombre homologado (reporte_ventas_DD_MM_YYYY), programarlo para generarse de lunes a viernes a las 08:00

In [1]:
!pip install openpyxl

#### Simulación de datos

In [2]:
import pandas as pd
import datetime as datetime
import numpy as np

In [11]:
# Simulando los datos de ventas

productos = ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E']
regiones = ['Norte', 'Centro', 'Oriente', 'Poniente', 'Sur']

df = pd.DataFrame({
    'Producto': np.random.choice(productos, size=50),
    'Región': np.random.choice(regiones, size=50),
    'Ventas': np.random.randint(100, 500, size=50),
})

In [12]:
df.head()

,Producto,Región,Ventas
0,Producto D,Sur,414
1,Producto D,Sur,257
2,Producto A,Centro,334
3,Producto B,Oriente,438
4,Producto C,Centro,459


#### Crear el excel con formato

In [27]:
# Fecha para el nombre del archivo
hoy = datetime.date.today().strftime('%d_%m_%Y')
nombre_archivo = f'reporte_ventas_{hoy}.xlsx'

# Crear un resumen por producto y región
resumen = df.groupby(['Producto', 'Región']).sum().reset_index()

# Crear el excel
with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='datos', index=False)
    resumen.to_excel(writer, sheet_name='Resumen', index=False)

    # Access the workbook and sheets
    workbook = writer.book
    worksheet = workbook['Resumen']

    # Formato a los encabezados using openpyxl
    from openpyxl.styles import Font, PatternFill, Alignment

    font = Font(bold=True)
    fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
    alignment = Alignment(horizontal='center', vertical='center')

    # Apply format to headers
    for col_num, value in enumerate(resumen.columns):
        cell = worksheet.cell(row=1, column=col_num + 1)
        cell.font = font
        cell.fill = fill
        cell.alignment = alignment

    # Ajustar ancho de columnas automaticamente
    for i, col in enumerate(resumen.columns):
        max_len = max(resumen[col].astype(str).map(len).max(), len(col))
        worksheet.column_dimensions[chr(65 + i)].width = max_len + 2


    # Create a new sheet for the chart
    worksheet_grafica = workbook.create_sheet('Grafica')

    # Import chart classes
    from openpyxl.chart import BarChart, Reference

    # Create a Bar chart
    chart = BarChart()

    # Add data to the chart
    data = Reference(worksheet, min_col=3, min_row=2, max_col=3, max_row=len(resumen) + 1)
    categories = Reference(worksheet, min_col=1, min_row=2, max_col=1, max_row=len(resumen) + 1)

    chart.add_data(data, titles_from_data=False)
    chart.set_categories(categories)

    # Set chart title and axis titles
    chart.title = "Ventas por Producto y Región"
    chart.x_axis.title = "Producto y Región"
    chart.y_axis.title = "Ventas"

    # Insert the chart into the 'Grafica' sheet
    worksheet_grafica.add_chart(chart, "B3")

### Automatizando con **schedule**

In [26]:
!pip install schedule

In [25]:
import schedule
import time

In [29]:
"""
def generar_reporte():
    print('Generando reporte de ventas diario')
    # Fecha para el nombre del archivo
    hoy = datetime.date.today().strftime('%d_%m_%Y')
    nombre_archivo = f'reporte_ventas_{hoy}.xlsx'

    # Crear un resumen por producto y región
    resumen = df.groupby(['Producto', 'Región']).sum().reset_index()

    # Crear el excel
    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='datos', index=False)
        resumen.to_excel(writer, sheet_name='Resumen', index=False)

        # Access the workbook and sheets
        workbook = writer.book
        worksheet = workbook['Resumen']

        # Formato a los encabezados using openpyxl
        from openpyxl.styles import Font, PatternFill, Alignment

        font = Font(bold=True)
        fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
        alignment = Alignment(horizontal='center', vertical='center')

        # Apply format to headers
        for col_num, value in enumerate(resumen.columns):
            cell = worksheet.cell(row=1, column=col_num + 1)
            cell.font = font
            cell.fill = fill
            cell.alignment = alignment

        # Ajustar ancho de columnas automaticamente
        for i, col in enumerate(resumen.columns):
            max_len = max(resumen[col].astype(str).map(len).max(), len(col))
            worksheet.column_dimensions[chr(65 + i)].width = max_len + 2


        # Create a new sheet for the chart
        worksheet_grafica = workbook.create_sheet('Grafica')

        # Import chart classes
        from openpyxl.chart import BarChart, Reference

        # Create a Bar chart
        chart = BarChart()

        # Add data to the chart
        data = Reference(worksheet, min_col=3, min_row=2, max_col=3, max_row=len(resumen) + 1)
        categories = Reference(worksheet, min_col=1, min_row=2, max_col=1, max_row=len(resumen) + 1)

        chart.add_data(data, titles_from_data=False)
        chart.set_categories(categories)

        # Set chart title and axis titles
        chart.title = "Ventas por Producto y Región"
        chart.x_axis.title = "Producto y Región"
        chart.y_axis.title = "Ventas"

        # Insert the chart into the 'Grafica' sheet
        worksheet_grafica.add_chart(chart, "B3")


#Ejecutar cada día a las 08:00
schedule.every().day.at("08:00").do(generar_reporte)

while True:
    schedule.run_pending()
    time.sleep(1)

"""

'\ndef generar_reporte():\n    print(\'Generando reporte de ventas diario\')\n    # Fecha para el nombre del archivo\n    hoy = datetime.date.today().strftime(\'%d_%m_%Y\')\n    nombre_archivo = f\'reporte_ventas_{hoy}.xlsx\'\n\n    # Crear un resumen por producto y región\n    resumen = df.groupby([\'Producto\', \'Región\']).sum().reset_index()\n\n    # Crear el excel\n    with pd.ExcelWriter(nombre_archivo, engine=\'openpyxl\') as writer:\n        df.to_excel(writer, sheet_name=\'datos\', index=False)\n        resumen.to_excel(writer, sheet_name=\'Resumen\', index=False)\n\n        # Access the workbook and sheets\n        workbook = writer.book\n        worksheet = workbook[\'Resumen\']\n\n        # Formato a los encabezados using openpyxl\n        from openpyxl.styles import Font, PatternFill, Alignment\n\n        font = Font(bold=True)\n        fill = PatternFill(start_color=\'F2F2F2\', end_color=\'F2F2F2\', fill_type=\'solid\')\n        alignment = Alignment(horizontal=\'center\'